In [ ]:
import os 
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

load_dotenv()

base_path = os.getenv("BASE_LOCAL_DIR_PATH")

: 

# Apply Pearson correlation coeifficient to users
---
- 주어진 신청&선정 내역 내, 특정 제품에 대한 동시평가 요소가 너무 적어 새로운 방식 적용 시도.
- 개괄적인 blueprint를 잡기 위한 내용으로, 쉽고 단순한 모델들을 활용해 pre-test 시도.

#### How to get PCC?
- 평점은 user별로 제공받은 제품 빈도 대비 선정율로 정의
- $x$축에는 `user_1` & $y$축에는 `user_2`를 가지는 2차원 평면에서 두 `user`간의 평점 사이에 상관관계를 탐색